# Mission to Mars (Data Scraping)

In [4]:
# All scraping and analysis
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import requests

In [5]:
#url list
url_mars_news = 'https://mars.nasa.gov/news/'
url_jpl_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url_mars_weather = 'https://twitter.com/marswxreport?lang=en'
url_mars_facts = 'https://space-facts.com/mars/'
url_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


## Mars News

In [6]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_mars_news)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_title = soup.find('div', class_="list_text").find('div', class_="content_title").get_text(strip=True)
news_p = soup.find('div', class_="article_teaser_body").get_text(strip=True)

print(news_title)
print('-------------------')
print(news_p)

NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
-------------------
Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.


## JPL Images

## Weather

## Facts

## Hemisphere Photos